### Hola mundo en LangChain

#### Instalar librerías pricipales y configuración de API Key de OpenAI

In [1]:
%%capture
!pip install langchain pypdf chromadb tiktoken

In [2]:
from getpass import getpass
import os

OPENAI_API_KEY = getpass('Enter the secret value: ')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

## Carga de documentos

In [4]:
import requests
from langchain.document_loaders import PyPDFLoader

urls = [
    'https://arxiv.org/pdf/2306.06031v1.pdf',
    'https://arxiv.org/pdf/2306.12156v1.pdf',
    'https://arxiv.org/pdf/2306.14289v1.pdf',
    'https://arxiv.org/pdf/2305.10973v1.pdf',
    'https://arxiv.org/pdf/2306.13643v1.pdf'
]

ml_papers = []

for i, url in enumerate(urls):
    response = requests.get(url)
    filename = f'paper{i+1}.pdf'
    with open(filename, 'wb') as f:
        f.write(response.content)
        print(f'Descargado {filename}')

        loader = PyPDFLoader(filename)
        data = loader.load()
        ml_papers.extend(data)

# Utiliza la lista ml_papers para acceder a los elementos de todos los documentos descargados
print('Contenido de ml_papers:')
print()
    

Descargado paper1.pdf
Descargado paper2.pdf
Descargado paper3.pdf
Descargado paper4.pdf
Descargado paper5.pdf
Contenido de ml_papers:



In [5]:
type(ml_papers), len(ml_papers), ml_papers[3]

(list,
 57,
 Document(page_content='Figure 1: FinGPT Framework.\n4.1 Data Sources\nThe first stage of the FinGPT pipeline involves the collec-\ntion of extensive financial data from a wide array of online\nsources. These include, but are not limited to:\n•Financial news: Websites such as Reuters, CNBC, Yahoo\nFinance, among others, are rich sources of financial news\nand market updates. These sites provide valuable informa-\ntion on market trends, company earnings, macroeconomic\nindicators, and other financial events.\n•Social media : Platforms such as Twitter, Facebook, Red-\ndit, Weibo, and others, offer a wealth of information in\nterms of public sentiment, trending topics, and immediate\nreactions to financial news and events.\n•Filings : Websites of financial regulatory authorities, such\nas the SEC in the United States, offer access to company\nfilings. These filings include annual reports, quarterly earn-\nings, insider trading reports, and other important company-\nspecific in

# Split documents

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
    length_function=len
)

documents = text_splitter.split_documents(ml_papers)

In [7]:
len(documents), documents[10]

(211,
 Document(page_content='highly volatile, changing rapidly in response to news events\nor market movements.\nTrends , often observable through websites like Seeking\nAlpha, Google Trends, and other finance-oriented blogs and\nforums, offer critical insights into market movements and in-\nvestment strategies. They feature:\n•Analyst perspectives: These platforms provide access to\nmarket predictions and investment advice from seasoned\nfinancial analysts and experts.\n•Market sentiment: The discourse on these platforms can\nreflect the collective sentiment about specific securities,\nsectors, or the overall market, providing valuable insights\ninto the prevailing market mood.\n•Broad coverage: Trends data spans diverse securities and\nmarket segments, offering comprehensive market coverage.\nEach of these data sources provides unique insights into\nthe financial world. By integrating these diverse data types,\nfinancial language models like FinGPT can facilitate a com-\nprehensive 

### Embeddings e ingesta a base de datos vectorial

In [2]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings
)

retriever = vectorstore.as_retriever(
    search_kwargs={"k": 3}
)

/home/tux/curso-langchain-chatbot/env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


ValidationError: 1 validation error for OpenAIEmbeddings
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

### Modelos de chat y cadenas para consulta de información

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

chat = ChatOpenAI(
    openai_ai_key=OPENAI_API_KEY,
    model_name='gpt-3.5-turbo',
    temperature=0.0
)

qa_chain = RetrievalQA.from_chain_type(
    llm=chat,
    chain_type='stuff',
    retriever=retriever
)

In [ ]:
query = "que es fingpt?"
qa_chain.run(query)

In [ ]:
query = "que es fast segment?"
qa_chain.run(query)

In [ ]:
query = "que es fast segment?"
qa_chain.run(query)